In [1]:
import argparse
import sys
import re
import os, glob, datetime, time
import numpy as np
import torch
import torch.nn as nn
from torch.nn.modules.loss import _Loss
import torch.nn.init as init
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
import data_generator as dg
from data_generator import DenoisingDataset
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# Params
if any(["jupyter" in arg for arg in sys.argv]):
    # Simulate command line arguments (replace these with your desired defaults)
    sys.argv = ['ipykernel_launcher.py', '--model', 'DnCNN', '--batch_size', '64', '--train_data', 'data/Train400', '--sigma', '4', '--epoch', '3', '--lr', '0.0005']

parser = argparse.ArgumentParser(description='PyTorch DnCNN')
parser.add_argument('--model', default='DnCNN', type=str, help='choose a type of model')
parser.add_argument('--batch_size', default=64, type=int, help='batch size')
parser.add_argument('--train_data', default='data/Train400', type=str, help='path of train data')
parser.add_argument('--sigma', default=4, type=int, help='noise level')
parser.add_argument('--epoch', default=3, type=int, help='number of train epoches')
parser.add_argument('--lr', default=5e-4, type=float, help='initial learning rate for Adam')
args = parser.parse_args()
batch_size = args.batch_size
cuda = torch.cuda.is_available()
n_epoch = args.epoch
sigma = args.sigma

save_dir = os.path.join('models', args.model+'_' + 'sigma' + str(sigma))

if not os.path.exists(save_dir):
    os.mkdir(save_dir)
        
        
class DnCNN(nn.Module):
    def __init__(self, out_channels=64, in_channels=1, kernel_size=3):
        super(DnCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1, stride=1)
        self.BN = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        output_layers = []
        for _ in range(10):
            for sub in self.make_Residual():
                output_layers.append(sub)
        self.out = nn.Sequential(*output_layers)
        self.res = nn.Conv2d(64, 1, kernel_size=3, padding=1, stride=1)
        
    def make_Residual(self, in_channels=64, out_channels=64, kernel_size=3):
        i = 0
        layers = []
        for i in range(2):
            layers.append(nn.Conv2d(64, 64, kernel_size=3, padding=1, stride=1, bias=True))
            layers.append(nn.BatchNorm2d(64, eps=0.0001, momentum=0.95))
            layers.append(nn.ReLU(inplace=True))
        return layers

    
    def forward(self, x):
        y = x
        out = self.relu(self.BN(self.conv1(x)))
        out = self.out(out)
        dn = y - self.res(out)
        return dn
        

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.orthogonal_(m.weight)
                print('init weight')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)

                
class sum_squared_error(_Loss):  # PyTorch 0.4.1
    """
    Definition: sum_squared_error = 1/2 * nn.MSELoss(reduction = 'sum')
    The backward is defined as: input-target
    """
    def __init__(self, size_average=None, reduce=None, reduction='sum'):
        super(sum_squared_error, self).__init__(size_average, reduce, reduction)

    def forward(self, input, target):
        # return torch.sum(torch.pow(input-target,2), (0,1,2,3)).div_(2)
        return torch.nn.functional.mse_loss(input, target, size_average=None, reduce=None, reduction='sum').div_(2)


def log(*args, **kwargs):
     print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S:"), *args, **kwargs)

In [4]:
import torch

# Attempt to manually select a CUDA device
try:
    torch.cuda.current_device()
    print("Current CUDA device:", torch.cuda.get_device_name())
except Exception as e:
    print("Error accessing CUDA device:", str(e))

Current CUDA device: NVIDIA GeForce GTX 1080 Ti


In [5]:
from tqdm import tqdm

if __name__ == '__main__':
    # model selection
    print('===> Building model')
    model = DnCNN()
    
    initial_epoch = 0
    model.train()
    # criterion = nn.MSELoss(reduction = 'sum')  # PyTorch 0.4.1
    criterion = sum_squared_error()
    if cuda:
        model = model.cuda()
         # device_ids = [0]
         # model = nn.DataParallel(model, device_ids=device_ids).cuda()
         # criterion = criterion.cuda()
    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay = 1e-4)
    scheduler = MultiStepLR(optimizer, milestones=[3, 6, 9], gamma=0.2)  # learning rates
    for epoch in tqdm(range(initial_epoch, n_epoch)):

        scheduler.step(epoch)  # step to the learning rate in this epcoh
        xs = dg.datagenerator(data_dir=args.train_data)
        xs = xs.astype('float32')/255.0
        xs = torch.from_numpy(xs.transpose((0, 3, 1, 2)))  # tensor of the clean patches, NXCXHXW
        DDataset = DenoisingDataset(xs, sigma)
        DLoader = DataLoader(dataset=DDataset, num_workers=4, drop_last=True, batch_size=batch_size, shuffle=True)
        epoch_loss = 0
        start_time = time.time()

        for n_count, batch_yx in enumerate(DLoader):
                optimizer.zero_grad()
                if cuda:
                    batch_x, batch_y = batch_yx[1].cuda(), batch_yx[0].cuda()
                loss = criterion(model(batch_y), batch_x)
                epoch_loss += loss.item()
                loss.backward()
                optimizer.step()
                if n_count % 10 == 0:
                    print('%4d %4d / %4d loss = %2.4f' % (epoch+1, n_count, xs.size(0)//batch_size, loss.item()/batch_size))
        elapsed_time = time.time() - start_time

        log('epcoh = %4d , loss = %4.4f , time = %4.2f s' % (epoch+1, epoch_loss/n_count, elapsed_time))
        np.savetxt('train_result.txt', np.hstack((epoch+1, epoch_loss/n_count, elapsed_time)), fmt='%2.4f')
        # torch.save(model.state_dict(), os.path.join(save_dir, 'model_%03d.pth' % (epoch+1)))
        torch.save(model, os.path.join(save_dir, 'model_%03d.pth' % (epoch+1)))


===> Building model


  0%|          | 0/3 [00:00<?, ?it/s]

^_^-training data finished-^_^
   1    0 / 2728 loss = 3662.9446
   1   10 / 2728 loss = 1309.7716
   1   20 / 2728 loss = 946.0906
   1   30 / 2728 loss = 547.5883
   1   40 / 2728 loss = 360.2235
   1   50 / 2728 loss = 284.0804
   1   60 / 2728 loss = 246.4255
   1   70 / 2728 loss = 209.9993
   1   80 / 2728 loss = 215.2713
   1   90 / 2728 loss = 166.1460
   1  100 / 2728 loss = 177.8282
   1  110 / 2728 loss = 127.0031
   1  120 / 2728 loss = 177.3469
   1  130 / 2728 loss = 102.6135
   1  140 / 2728 loss = 90.2536
   1  150 / 2728 loss = 117.1431
   1  160 / 2728 loss = 100.8194
   1  170 / 2728 loss = 71.7937
   1  180 / 2728 loss = 65.3173
   1  190 / 2728 loss = 79.4943
   1  200 / 2728 loss = 71.9804
   1  210 / 2728 loss = 71.8738
   1  220 / 2728 loss = 56.3057
   1  230 / 2728 loss = 68.4615
   1  240 / 2728 loss = 52.0838
   1  250 / 2728 loss = 57.5822
   1  260 / 2728 loss = 50.1758
   1  270 / 2728 loss = 64.8028
   1  280 / 2728 loss = 50.7798
   1  290 / 2728 loss =

 33%|███▎      | 1/3 [03:52<07:45, 232.59s/it]

2024-12-13 02:30:24: epcoh =    1 , loss = 3548.9169 , time = 230.53 s
^_^-training data finished-^_^
   2    0 / 2728 loss = 17.1914
   2   10 / 2728 loss = 16.1395
   2   20 / 2728 loss = 10.9203
   2   30 / 2728 loss = 19.1820
   2   40 / 2728 loss = 38.5931
   2   50 / 2728 loss = 12.7421
   2   60 / 2728 loss = 24.0554
   2   70 / 2728 loss = 23.9911
   2   80 / 2728 loss = 8.3231
   2   90 / 2728 loss = 11.0193
   2  100 / 2728 loss = 12.4186
   2  110 / 2728 loss = 11.8963
   2  120 / 2728 loss = 9.0626
   2  130 / 2728 loss = 10.4495
   2  140 / 2728 loss = 25.0496
   2  150 / 2728 loss = 13.2314
   2  160 / 2728 loss = 11.9627
   2  170 / 2728 loss = 10.2880
   2  180 / 2728 loss = 12.3104
   2  190 / 2728 loss = 13.9772
   2  200 / 2728 loss = 26.4489
   2  210 / 2728 loss = 26.8817
   2  220 / 2728 loss = 35.2934
   2  230 / 2728 loss = 30.3697
   2  240 / 2728 loss = 11.0570
   2  250 / 2728 loss = 12.4976
   2  260 / 2728 loss = 13.2533
   2  270 / 2728 loss = 30.1403
   2

 67%|██████▋   | 2/3 [07:45<03:52, 232.67s/it]

2024-12-13 02:34:17: epcoh =    2 , loss = 1172.7178 , time = 230.71 s
^_^-training data finished-^_^
   3    0 / 2728 loss = 8.9296
   3   10 / 2728 loss = 15.9136
   3   20 / 2728 loss = 7.1544
   3   30 / 2728 loss = 5.5564
   3   40 / 2728 loss = 20.4171
   3   50 / 2728 loss = 6.8679
   3   60 / 2728 loss = 43.5309
   3   70 / 2728 loss = 22.3931
   3   80 / 2728 loss = 32.0310
   3   90 / 2728 loss = 10.8197
   3  100 / 2728 loss = 26.5948
   3  110 / 2728 loss = 21.4885
   3  120 / 2728 loss = 16.5310
   3  130 / 2728 loss = 7.2722
   3  140 / 2728 loss = 12.1117
   3  150 / 2728 loss = 11.6148
   3  160 / 2728 loss = 13.6156
   3  170 / 2728 loss = 28.6625
   3  180 / 2728 loss = 8.1206
   3  190 / 2728 loss = 11.8085
   3  200 / 2728 loss = 9.8870
   3  210 / 2728 loss = 9.4762
   3  220 / 2728 loss = 5.4618
   3  230 / 2728 loss = 8.6192
   3  240 / 2728 loss = 15.2850
   3  250 / 2728 loss = 7.9141
   3  260 / 2728 loss = 18.2620
   3  270 / 2728 loss = 18.9702
   3  280 / 2

100%|██████████| 3/3 [11:38<00:00, 232.73s/it]

2024-12-13 02:38:09: epcoh =    3 , loss = 885.4382 , time = 230.83 s
